In [15]:
import os
import sys
import cv2
import h5py
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from datetime import datetime
from tqdm import tqdm
from utils import get_params_count

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.applications import inception_v3, xception, resnet50, vgg16, vgg19
from keras.applications import InceptionV3, Xception, ResNet50, VGG16, VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, Dense, Dropout, Activation, Flatten, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Model
from keras.optimizers import SGD

In [54]:
height = 224
train_name = os.listdir('./train')
test_name = os.listdir('./test')
train_size = len(train_name)
test_size = len(test_name)
train = np.zeros((train_size, height, height, 3), dtype=np.float32)
test = np.zeros((test_size, height, height, 3), dtype=np.float32)
print("Train: %d, Test: %d" % (train_size, test_size))

for i in tqdm(range(train_size)):
    img = cv2.imread('./train/%s.jpg' % labels['id'][i])
    img = cv2.resize(img, dsize=(height, height))
    img = img[:, :, ::-1]
    train[i] = img
    
for i in tqdm(range(test_size)):
    img = cv2.imread('./test/%s.jpg' % test_labels['id'][i])
    img = cv2.resize(img, dsize=(height, height))
    img = img[:, :, ::-1]
    test[i] = img
    
print('Training Data Size = %.2f GB' % (sys.getsizeof(train)/1024**3))
print('Testing Data Size = %.2f GB' % (sys.getsizeof(test)/1024**3))

Train: 10222, Test: 10357


100%|█████████████████████████████████████████████████████| 10357/10357 [00:27<00:00, 381.74it/s]


Training Data Size = 5.73 GB
Testing Data Size = 5.81 GB


In [18]:
train = resnet50.preprocess_input(train)
test = resnet50.preprocess_input(test)

In [11]:
labels = pd.read_csv('labels.csv')
test_labels = pd.read_csv('sample_submission.csv')

breeds = list(set(labels['breed']))
breeds.sort()
print("Total Dog Breeds:", len(breeds))

Y_train = np.zeros((train_size, len(breeds)), dtype=np.uint8)
for i in range(train_size):
    onehot = breeds.index(labels['breed'][i])
    Y_train[i][onehot] = 1

Total Dog Breeds: 120


In [12]:
def export_gap(MODEL, preprocess=None, batch_size=128):
    x = Input(shape=(height, height, 3))
    if preprocess is not None:
        x = Lambda(preprocess)(x)
    model = MODEL(include_top=False, input_tensor=x, weights='imagenet', pooling='avg')
    train_gap = model.predict(train, batch_size=batch_size)
    test_gap = model.predict(test, batch_size=batch_size)
    with h5py.File("gap_%s.h5" % MODEL.__name__, 'w') as f:
        f.create_dataset('train', data=train_gap)
        f.create_dataset('test', data=test_gap)

In [13]:
export_gap(InceptionV3, inception_v3.preprocess_input)

In [14]:
export_gap(Xception, xception.preprocess_input, 64)

In [16]:
export_gap(InceptionResNetV2, inception_v3.preprocess_input)

In [56]:
export_gap(ResNet50)

## 训练

In [65]:
train_temp = []
test_temp = []
# 'gap_InceptionV3.h5', 'gap_Xception.h5', 'gap_ResNet50.h5', 'gap_InceptionResNetV2.h5'
for gapfile in ['gap_InceptionV3.h5', 'gap_Xception.h5', 'gap_InceptionResNetV2.h5']:
    with h5py.File(gapfile, 'r') as f:
        train_temp.append(np.array(f['train']))
        test_temp.append(np.array(f['test']))
train_gap = np.concatenate(train_temp, axis=1)
test_gap = np.concatenate(test_temp, axis=1)
print("Number of Features:", train_gap.shape[1])

Number of Features: 5632


In [66]:
X_train, X_val, y_train, y_val = train_test_split(train_gap, Y_train, shuffle=True, test_size=0.2, random_state=42)

In [83]:
# Input Shape: (Batch Size, Feature Vector length)
x = Input(shape=(X_train.shape[1],))
y = Dropout(0.2)(x)
y = Dense(120, activation='softmax', kernel_initializer='he_normal', name='classifier')(y)
model_gap = Model(inputs=x, outputs=y, name='GAP')
model_gap.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
print('Trainable: %d, Non-Trainable: %d' % get_params_count(model_gap))

Trainable: 675960, Non-Trainable: 0


In [84]:
# Prepare Callbacks for Model Checkpoint, Early Stopping and Tensorboard.
log_name = '/DogBreed-EP{epoch:02d}-LOSS{val_loss:.4f}.h5'
log_dir = datetime.now().strftime('gap_model_%Y%m%d_%H%M')
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(log_dir + log_name, monitor='val_loss', save_best_only=True)
tb = TensorBoard(log_dir=log_dir)

model_gap.fit(x=X_train, y=y_train, batch_size=16, epochs=20, validation_data=(X_val, y_val), callbacks=[es, mc, tb])

# model_gap.fit(x=train_gap, y=Y_train, shuffle=True, batch_size=16, epochs=5)

Epoch 1/5
10222/10222 [==============================] - 3s - loss: 0.7017 - acc: 0.8620     
Epoch 2/5
10222/10222 [==============================] - 2s - loss: 0.2120 - acc: 0.9332     
Epoch 3/5
10222/10222 [==============================] - 2s - loss: 0.1865 - acc: 0.9395     
Epoch 4/5
10222/10222 [==============================] - 2s - loss: 0.1738 - acc: 0.9452     
Epoch 5/5
10222/10222 [==============================] - 2s - loss: 0.1674 - acc: 0.9464     


## 导出

In [76]:
model_gap.load_weights('./gap_model_20171015_2231/DogBreed-EP08-LOSS0.2017.h5')

In [85]:
y_test = model_gap.predict(test_gap)

In [86]:
test_labels.iloc[:, 1:] = y_test
test_labels.to_csv('results.csv', index=False)